
<img src="https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png" style="float: left;"> 

# Lesson 6.03 - Random Forests and ExtraTrees

## Random Forests

With bagged decision trees, we generate many different trees on pretty similar data. These trees are **strongly correlated** with one another. Because these trees are correlated with one another, they will have high variance. Looking at the variance of the average of two random variables $T_1$ and $T_2$:

$$
\begin{eqnarray*}
Var\left(\frac{T_1+T_2}{2}\right) &=& \frac{1}{4}\left[Var(T_1) + Var(T_2) + 2Cov(T_1,T_2)\right]
\end{eqnarray*}
$$

If $T_1$ and $T_2$ are highly correlated, then the variance will about as high as we'd see with individual decision trees. By "de-correlating" our trees from one another, we can drastically reduce the variance of our model.

That's the difference between bagged decision trees and random forests! We're going to do the same thing as before, but we're going to de-correlate our trees. This will reduce our variance (at the expense of a small increase in bias) and thus should greatly improve the overall performance of the final model.

### So how do we "de-correlate" our trees?
Random forests differ from bagging decision trees in only one way: they use a modified tree learning algorithm that selects, at each split in the learning process, a **random subset of the features**. This process is sometimes called the *random subspace method*.

The reason for doing this is the correlation of the trees in an ordinary bootstrap sample: if one or a few features are very strong predictors for the response variable (target output), these features will be used in many/all of the bagged decision trees, causing them to become correlated. By selecting a random subset of features at each split, we counter this correlation between base trees, strengthening the overall model.

For a problem with $p$ features, it is typical to use:

- $\sqrt{p}$ (rounded down) features in each split for a classification problem.
- $p/3$ (rounded down) with a minimum node size of 5 as the default for a regression problem.

While this is a guideline, Hastie and Tibshirani (authors of Introduction to Statistical Learning and Elements of Statistical Learning) have suggested this as a good rule in the absence of some rationale to do something different.

Random forests, a step beyond bagged decision trees, are **very widely used** classifiers and regressors. They are relatively simple to use because they require very few parameters to set and they perform pretty well.
- It is quite common for interviewers to ask how a random forest is constructed or how it is superior to a single decision tree.

--- 

## Extremely Randomized Trees (ExtraTrees)
Adding one more step of randomization (and thus de-correlation) yields extremely randomized trees, or _ExtraTrees_. These are trained using bagging (sampling of observations) and the random subspace method (sampling of features), but an additional layer of randomness is introduced. Instead of computing the locally optimal feature/split combination (based on, e.g., information gain or the Gini impurity) for each feature under consideration, a random value is selected for the split. This value is selected from the feature's empirical range.

This further reduces the variance, but causes an increase in bias. If you're considering using ExtraTrees, you might consider this to be a hyperparameter you can tune. Build an ExtraTrees model and a Random Forest model, then compare their performance!

That's exactly what we'll do below.

## Import libraries

We'll need the following libraries for today's lecture:
- `pandas`
- `numpy`
- `GridSearchCV`, `train_test_split` and `cross_val_score` from `sklearn`'s `model_selection` module 
- `RandomForestClassifier` and `ExtraTreesClassifier` from `sklearn`'s `ensemble` module 

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

## Load Data

Load `train.csv` and `test.csv` from Kaggle into `DataFrames`.

In [6]:
train = pd.read_csv('../train.csv')
test = pd.read_csv('../test.csv')

## Data Cleaning: Drop the two rows with missing `Embarked` values from train

In [7]:
train = train[~train['Embarked'].isnull()].copy()

## Data Cleaning: `Fare`

The test set has one row with a missing value for `Fare`. Fill it with the average `Fare` with everyone from the same `Pclass`. **Use the training set to calculate the average!**

In [8]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [9]:
test[test['Fare'].isnull()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S


In [10]:
avg_fare = train.loc[train['Pclass'] == 3, 'Fare'].mean()
test['Fare'] = test['Fare'].fillna(avg_fare)

## Data Cleaning: `Age`

Let's simply impute all missing ages to be **999**. 

**NOTE**: This is not a best practice. However, 
1. Since we haven't really covered imputation in depth
2. And the proper way would take too long to implement (thus detracting) from today's lecture
3. And since we're ensembling with Decision Trees

We'll do it this way as a matter of convenience.

In [11]:
test['Age'] = test['Age'].fillna(999)
train['Age'] = train['Age'].fillna(999)

## Feature Engineering: Extract `Title` from `Name`

In [12]:
train['Title'] = train['Name'].str.extract(r',\s([^\.]+)', expand=False)
test['Title'] = test['Name'].str.extract(r',\s([^\.]+)', expand=False)

In [13]:
set(list(train['Title'].unique()) + list(test['Title'].unique()))

{'Capt',
 'Col',
 'Don',
 'Dona',
 'Dr',
 'Jonkheer',
 'Lady',
 'Major',
 'Master',
 'Miss',
 'Mlle',
 'Mme',
 'Mr',
 'Mrs',
 'Ms',
 'Rev',
 'Sir',
 'the Countess'}

In [14]:
def bucket_title(title):
    if title in ['Capt', 'Col', 'Major']:
        return 'Military Nobility'
    if title in ['Miss', 'Mlle']:
        return 'Miss'
    if title in ['Mrs', 'Mme']:
        return 'Mrs'
    if title == 'Master':
        return title
    return 'Civilian Nobility'
train['Title'] = train['Title'].map(bucket_title)
test['Title'] = test['Title'].map(bucket_title)

In [15]:
train['Title'].isnull().sum()

0

In [16]:
test['Title'].isnull().sum()

0

## Feature Engineering: `Cabin`

Since there are so many missing values for `Cabin`, let's binarize that column as follows:
- 1 if there originally was a value for `Cabin`
- 0 if it was null

**Do this for both `train` and `test`**

In [17]:
train['Cabin'] = (~train['Cabin'].isnull()).astype(int)
test['Cabin'] = (~test['Cabin'].isnull()).astype(int)

## Feature Engineering: Dummies

Dummy the `Sex`, `Embarked` and `Title` columns. Be sure to set `drop_first=True`.

In [18]:
dum_cols = ['Sex', 'Embarked', 'Title']
train = pd.get_dummies(train, columns=dum_cols, drop_first=True)
test = pd.get_dummies(test, columns=dum_cols, drop_first=True)

In [19]:
train.head()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex_male,Embarked_Q,Embarked_S,Title_Master,Title_Military Nobility,Title_Miss,Title_Mrs
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,0,1,0,1,0,0,0,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,1,0,0,0,0,0,0,1
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,0,0,0,1,0,0,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,1,0,0,1,0,0,0,1
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,0,1,0,1,0,0,0,0


## Model Prep: Create `X` and `y` variables

Our features will be all the `Title` columns plus:
- `Pclass`
- `Age`
- `SibSp`
- `Parch`
- `Fare`
- `Cabin`
- `Sex_male`
- `Embarked_Q`
- `Embarked_S`

And our target will be `Survived`

In [21]:
features = [col for col in train.columns if 'Title' in col]
features

['Title_Master', 'Title_Military Nobility', 'Title_Miss', 'Title_Mrs']

In [22]:
features.extend(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Sex_male', 'Embarked_Q', 'Embarked_S'])
X = train[features]
y = train['Survived']

## Callenge: What is our baseline accuracy?

In [23]:
y.value_counts(normalize=True)

0    0.617548
1    0.382452
Name: Survived, dtype: float64

## Train/Test Split

I know it can be confusing having an `X_test` from our training data vs a test set from Kaggle. If you want, you can use `X_val`/`y_val` for what we normally call `X_test`/`y_test`.

In [24]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42, stratify=y)

## Model instantiation

Create an instance of `RandomForestClassifier` and `ExtraTreesClassifier`.

In [25]:
rf = RandomForestClassifier()
et = ExtraTreesClassifier()

## Model Evaluation

Which one has a higher `cross_val_score`?

In [26]:
cross_val_score(rf, X_train, y_train).mean()

0.8033033033033034

In [27]:
cross_val_score(et, X_train, y_train).mean()

0.7912912912912913

## Grid Search

They're both pretty close performance-wise. We could Grid Search over both, but for the sake of time we'll go with `RandomForestClassifier`.

In [28]:
np.sqrt(len(features))

3.605551275463989

In [29]:
np.log2(len(features))

3.700439718141092

In [30]:
rf = RandomForestClassifier()
rf_params = {
    'n_estimators': [10, 20, 30],
    'max_depth': [None, 1, 2, 3, 4, 5],
    'min_samples_split': [2,3,4]
}
gs = GridSearchCV(rf, param_grid=rf_params)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

0.8153153153153153


{'max_depth': 4, 'min_samples_split': 4, 'n_estimators': 30}

In [31]:
gs.score(X_train, y_train)

0.8393393393393394

In [32]:
gs.score(X_val, y_val)

0.7892376681614349

## Kaggle Submission

Now that we've evaluated our model, let's submit our predictions to Kaggle.

In [33]:
pred = gs.predict(test[features])

In [34]:
test['Survived'] = pred
test[['PassengerId', 'Survived']].to_csv('submission.csv', index=False)